In [77]:
import pandas as pd
from geopy.distance import vincenty
import time

In [2]:
data_old = pd.read_excel(r"F:/temp/190408/1812JDlnglat.xlsx")
data_old.shape

(39258, 8)

In [3]:
data_new = pd.read_excel(r"F:/temp/190408/1902JDlnglat.xlsx")
data_new.shape

(70243, 8)

In [4]:
data_new.columns

Index(['订单编号', '承运商', 're_province_gaode', 're_city_gaode',
       're_district_gaode', 're_siteLng_gaode', 're_siteLat_gaode',
       're_location_gaode'],
      dtype='object')

In [5]:
df_old = data_old.copy()
df_old.shape

(39258, 8)

In [6]:
df_new = data_new.copy()
df_new.shape

(70243, 8)

In [7]:
df_old.head()

,订单编号,承运商,re_province_gaode,re_city_gaode,re_district_gaode,re_siteLng_gaode,re_siteLat_gaode,re_location_gaode
0,690000000001008318120420508973,京东线下,江苏省,南京市,江宁区,118.809944,31.943942,"118.809944,31.943942"
1,690000000001008318120420508818,京东线下,湖北省,咸宁市,通山县,114.494307,29.601624,"114.494307,29.601624"
2,690000000001008318120420508726,京东线下,江苏省,苏州市,张家港市,120.553559,31.872070,"120.553559,31.872070"
3,690000000001008318120420508594,京东线下,云南省,曲靖市,师宗县,103.919803,24.699084,"103.919803,24.699084"
4,690000000001008318120420508550,京东线下,河南省,郑州市,中原区,113.497570,34.765543,"113.497570,34.765543"


In [8]:
print(df_old["re_district_gaode"].value_counts())

余杭区              1137
江干区              1067
[]               1059
萧山区              1034
西湖区               823
洪山区               802
官渡区               768
金水区               534
五华区               524
滨江区               495
盘龙区               459
西山区               428
拱墅区               391
昆山市               351
麒麟区               347
呈贡区               321
吴中区               308
江夏区               291
下城区               287
中原区               273
武昌区               263
管城回族区             263
富阳区               254
新郑市               241
昭阳区               228
大理市               224
二七区               221
中牟县               214
黄陂区               207
江岸区               205
                 ... 
五峰土家族自治县            7
镇沅彝族哈尼族拉祜族自治县       7
沧源佤族自治县             7
吉利区                 6
梁河县                 6
点军区                 6
阳新县                 5
屏边苗族自治县             5
潜江市                 5
磐安县                 5
沁阳市                 5
解放区                 5
泰顺县                 4
贡山独龙族怒族自治县          4
宁蒗彝族自治县   

In [9]:
df_old = df_old[~df_old["re_district_gaode"].str.contains("\[\]",regex=True)]
print(df_old["re_district_gaode"].value_counts())

余杭区           1137
江干区           1067
萧山区           1034
西湖区            823
洪山区            802
官渡区            768
金水区            534
五华区            524
滨江区            495
盘龙区            459
西山区            428
拱墅区            391
昆山市            351
麒麟区            347
呈贡区            321
吴中区            308
江夏区            291
下城区            287
中原区            273
武昌区            263
管城回族区          263
富阳区            254
新郑市            241
昭阳区            228
大理市            224
二七区            221
中牟县            214
黄陂区            207
江岸区            205
汉阳区            204
              ... 
遂昌县              7
高港区              7
福贡县              7
点军区              6
梁河县              6
吉利区              6
屏边苗族自治县          5
沁阳市              5
磐安县              5
潜江市              5
解放区              5
阳新县              5
贡山独龙族怒族自治县       4
铁山区              4
泰顺县              4
宁蒗彝族自治县          4
景宁畲族自治县          3
庆元县              3
温县               3
新野县              3
岱山县              3
博爱县         

In [10]:
df_new = df_new[~df_new["re_district_gaode"].str.contains("\[\]",regex=True)]
print(df_new["re_district_gaode"].value_counts())

洪山区           1220
金水区            922
吴中区            721
昆山市            707
官渡区            580
余杭区            575
萧山区            528
江夏区            515
管城回族区          497
武昌区            486
江干区            477
中原区            455
江岸区            415
二七区            398
五华区            391
黄陂区            390
西湖区            373
盘龙区            371
江阴市            363
中牟县            352
新郑市            351
江宁区            346
武进区            344
吴江区            344
鼓楼区            341
汉阳区            340
西山区            338
江汉区            328
宛城区            307
涧西区            306
              ... 
墨江哈尼族自治县        15
陇川县             15
兴山县             15
点军区             15
福贡县             14
云龙县             14
五峰土家族自治县        13
宁洱哈尼族彝族自治县      13
河口瑶族自治县         13
庆元县             13
绿春县             13
景宁畲族自治县         13
水富县             12
铁山区             12
维西傈僳族自治县        12
修武县             11
永仁县             10
江城哈尼族彝族自治县      10
漾濞彝族自治县          9
石龙区              9
云和县              8
山城区         

In [46]:
print(df_new["承运商"].value_counts())

京东线下    61275
京东物流     7121
Name: 承运商, dtype: int64


In [65]:
sheng_list = df_old["re_district_gaode"].unique()
len(sheng_list)

570

In [96]:
def FenSheng(df_new,df_old,col_province,col_city,col_district,col_lng,col_lat):
    start_time = time.time()
    dfA = pd.DataFrame(columns=['orderID_old','orderID_new','distance'])
    limit_KM = round(3 /(3.14 * 6400 /180),6)
    
    
    province_list = df_old[col_province].unique()
    for sheng in province_list:
#         print(sheng)
#         print("*"*30)
        dfold_sheng = df_old[df_old[col_province] == sheng].reset_index(drop=True)
        dfnew_sheng = df_new[df_new[col_province] == sheng].reset_index(drop=True)
        
               
        city_list = dfold_sheng[col_city].unique()
        for shi in city_list:
#             print(sheng+shi)
#             print("*"*30)
            dfold_shi = dfold_sheng[(dfold_sheng[col_city] == shi)].reset_index(drop=True)
            dfnew_shi = dfnew_sheng[(dfnew_sheng[col_city] == shi)].reset_index(drop=True)   
            
            district_list = dfold_shi[col_district].unique()  
            print(district_list)
            for xian in district_list:
#                 print(sheng+shi+xian)
#                 print("*"*30)
                dfold = dfold_shi[dfold_shi[col_district] == xian].reset_index(drop=True)
                dfnew = dfnew_shi[dfnew_shi[col_district] == xian].reset_index(drop=True)
                
                print(sheng+shi+xian)
# #                 print(dfnew[0:1])

#                 if dfnew.empty == True:
#                     print(sheng + shi + xian + "：为空")
#                     pass
#                 else:
#                     for i in range(0,len(dfnew[col_lng])):
#                         for j in range(0,len(dfold[col_lng])):
# #                             print(len(dfnew[col_lng]) * len(dfold[col_lng]))
# #                             print("************************")
#                             start_lng = round(dfnew[col_lng][i],6)
#                             start_lat = round(dfnew[col_lat][i],6)
#                             end_lng = round(dfold[col_lng][j],6)
#                             end_lat = round(dfold[col_lat][j],6)
#                             start_lnglat = (start_lat,start_lng)
#                             end_lnglat = (end_lat,end_lng)
#                             if (abs(start_lat - end_lat) <= limit_KM or abs(start_lng - end_lng) <= limit_KM ):
#                                 a = vincenty(start_lnglat, end_lnglat).km
#                                 row={'orderID_old':dfold['订单编号'][j],'orderID_new':dfnew['订单编号'][i],'distance':a}
#                                 dfA = dfA.append(row,ignore_index=True)
#                             else:
#                                 pass
#                     print(str(sheng) + str(shi) + str(xian) + "：已完成")
#             end_time = time.time()
#             print("总用时：",end_time-start_time)
#             return dfA

In [97]:
# listA = []
# for i in range(0,len(df_new_sheng['re_location_gaode'])):
#     for j in range(0,len(df_old_sheng['re_location_gaode'])):
#         start_lng = round(df_new_sheng['re_siteLng_gaode'][i],6)
#         start_lat = round(df_new_sheng['re_siteLat_gaode'][j],6)
#         end_lng = round(df_old_sheng['re_siteLng_gaode'][j],6)
#         end_lat = round(df_old_sheng['re_siteLat_gaode'][j],6)
#         start_lnglat = (start_lat,start_lng)
#         end_lnglat = (end_lat,end_lng)
#         a = vincenty(start_lnglat, end_lnglat).km
#         listA.append(a)

In [98]:
listA = FenSheng(df_new,df_old,"re_province_gaode","re_city_gaode","re_district_gaode","re_siteLng_gaode","re_siteLat_gaode")

['江宁区' '建邺区' '玄武区' '高淳区' '雨花台区' '栖霞区' '浦口区' '六合区' '秦淮区' '鼓楼区' '溧水区']
江苏省南京市江宁区
江苏省南京市建邺区
江苏省南京市玄武区
江苏省南京市高淳区
江苏省南京市雨花台区
江苏省南京市栖霞区
江苏省南京市浦口区
江苏省南京市六合区
江苏省南京市秦淮区
江苏省南京市鼓楼区
江苏省南京市溧水区
['张家港市' '常熟市' '虎丘区' '相城区' '昆山市' '吴中区' '吴江区' '太仓市' '姑苏区']
江苏省苏州市张家港市
江苏省苏州市常熟市
江苏省苏州市虎丘区
江苏省苏州市相城区
江苏省苏州市昆山市
江苏省苏州市吴中区
江苏省苏州市吴江区
江苏省苏州市太仓市
江苏省苏州市姑苏区
['如皋市' '港闸区' '通州区' '如东县' '海安县' '海门市' '启东市' '崇川区']
江苏省南通市如皋市
江苏省南通市港闸区
江苏省南通市通州区
江苏省南通市如东县
江苏省南通市海安县
江苏省南通市海门市
江苏省南通市启东市
江苏省南通市崇川区
['京口区' '丹阳市' '句容市' '润州区' '扬中市' '丹徒区']
江苏省镇江市京口区
江苏省镇江市丹阳市
江苏省镇江市句容市
江苏省镇江市润州区
江苏省镇江市扬中市
江苏省镇江市丹徒区
['锡山区' '江阴市' '惠山区' '新吴区' '滨湖区' '梁溪区' '宜兴市']
江苏省无锡市锡山区
江苏省无锡市江阴市
江苏省无锡市惠山区
江苏省无锡市新吴区
江苏省无锡市滨湖区
江苏省无锡市梁溪区
江苏省无锡市宜兴市
['沭阳县' '泗阳县' '宿城区' '泗洪县' '宿豫区']
江苏省宿迁市沭阳县
江苏省宿迁市泗阳县
江苏省宿迁市宿城区
江苏省宿迁市泗洪县
江苏省宿迁市宿豫区
['靖江市' '海陵区' '兴化市' '泰兴市' '高港区' '姜堰区']
江苏省泰州市靖江市
江苏省泰州市海陵区
江苏省泰州市兴化市
江苏省泰州市泰兴市
江苏省泰州市高港区
江苏省泰州市姜堰区
['灌南县' '海州区' '赣榆区' '东海县' '灌云县' '连云区']
江苏省连云港市灌南县
江苏省连云港市海州区
江苏省连云港市赣榆区
江苏省连云港市东海县
江苏省连云港市灌云县
江苏省连云港市连云区
['射阳县' '阜宁县' '盐都区' '大丰区' '亭湖区' '响水县' '建湖县' '东台

['红旗区' '长垣县' '延津县' '原阳县' '新乡县' '封丘县' '辉县市' '卫滨区' '卫辉市' '凤泉区' '牧野区' '获嘉县']
河南省新乡市红旗区
河南省新乡市长垣县
河南省新乡市延津县
河南省新乡市原阳县
河南省新乡市新乡县
河南省新乡市封丘县
河南省新乡市辉县市
河南省新乡市卫滨区
河南省新乡市卫辉市
河南省新乡市凤泉区
河南省新乡市牧野区
河南省新乡市获嘉县
['淇滨区' '鹤山区' '山城区' '浚县' '淇县']
河南省鹤壁市淇滨区
河南省鹤壁市鹤山区
河南省鹤壁市山城区
河南省鹤壁市浚县
河南省鹤壁市淇县
['山阳区' '解放区' '温县' '武陟县' '沁阳市' '博爱县' '孟州市' '修武县' '中站区']
河南省焦作市山阳区
河南省焦作市解放区
河南省焦作市温县
河南省焦作市武陟县
河南省焦作市沁阳市
河南省焦作市博爱县
河南省焦作市孟州市
河南省焦作市修武县
河南省焦作市中站区
['济源市']
河南省[]济源市
['余杭区' '拱墅区' '萧山区' '淳安县' '富阳区' '滨江区' '上城区' '江干区' '西湖区' '临安区' '建德市' '下城区'
 '桐庐县']
浙江省杭州市余杭区
浙江省杭州市拱墅区
浙江省杭州市萧山区
浙江省杭州市淳安县
浙江省杭州市富阳区
浙江省杭州市滨江区
浙江省杭州市上城区
浙江省杭州市江干区
浙江省杭州市西湖区
浙江省杭州市临安区
浙江省杭州市建德市
浙江省杭州市下城区
浙江省杭州市桐庐县
['温岭市' '黄岩区' '天台县' '临海市' '椒江区' '玉环市' '仙居县' '路桥区' '三门县']
浙江省台州市温岭市
浙江省台州市黄岩区
浙江省台州市天台县
浙江省台州市临海市
浙江省台州市椒江区
浙江省台州市玉环市
浙江省台州市仙居县
浙江省台州市路桥区
浙江省台州市三门县
['南湖区' '桐乡市' '秀洲区' '嘉善县' '平湖市' '海宁市' '海盐县']
浙江省嘉兴市南湖区
浙江省嘉兴市桐乡市
浙江省嘉兴市秀洲区
浙江省嘉兴市嘉善县
浙江省嘉兴市平湖市
浙江省嘉兴市海宁市
浙江省嘉兴市海盐县
['莲都区' '遂昌县' '缙云县' '龙泉市' '松阳县' '庆元县' '景宁畲族自治县' '青田县' '云和县']
浙江省丽水市莲都区
浙江省丽水市遂昌县
浙江省丽水市缙云县
浙江省丽水市

In [70]:
len(listA)

TypeError: object of type 'NoneType' has no len()

In [56]:
df_new[df_new["re_province_gaode"] == "江苏省"]

,订单编号,承运商,re_province_gaode,re_city_gaode,re_district_gaode,re_siteLng_gaode,re_siteLat_gaode,re_location_gaode
4,600101844000008319020200931978,京东线下,江苏省,苏州市,太仓市,121.037769,31.527201,"121.037769,31.527201"
8,609903837000008319020801202801,京东线下,江苏省,苏州市,昆山市,121.054839,31.287773,"121.054839,31.287773"
12,690000000001008319022622247909,京东线下,江苏省,苏州市,太仓市,121.253560,31.578968,"121.253560,31.578968"
1308,690000000001008319022422204443,京东线下,江苏省,苏州市,相城区,120.629897,31.497427,"120.629897,31.497427"
6501,690000000001008319020721846449,京东线下,江苏省,苏州市,吴中区,120.807905,31.353657,"120.807905,31.353657"
14309,609904841000008319020601121224,京东线下,江苏省,苏州市,昆山市,120.957426,31.178496,"120.957426,31.178496"
15002,690000000001008319021522021076,京东线下,江苏省,苏州市,昆山市,121.017147,31.309667,"121.017147,31.309667"
20454,690000000001008319021822065369,京东线下,江苏省,南京市,栖霞区,118.870551,32.152524,"118.870551,32.152524"
20455,690000000001008319021822065360,京东线下,江苏省,南通市,海门市,121.414079,31.940197,"121.414079,31.940197"
20456,690000000001008319021822065162,京东线下,江苏省,无锡市,梁溪区,120.296140,31.583004,"120.296140,31.583004"


In [16]:
listA.shape

(59009, 3)

In [17]:
listB = listA[listA['distance'] <= 3]

In [18]:
listB.shape

(22123, 3)

In [19]:
listB.to_excel(r"F:\temp\190408\1812_1902_lnglat_dis.xlsx",index=False)